In [1]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
import os
os.chdir("C:/Users/Madhu/NLP/Senti")
pos_data = "pos.txt"
neg_data = "neg.txt"

In [3]:
max_lines=100000

In [4]:
def create_lexicon(pos,neg):
    
    lexicon=[]
    # read files
    with open(pos,"r") as pos_f:
        lines = pos_f.readlines()
        
        #for every line add tokens to lexicon
        for line in lines[:max_lines]:
            tokens=word_tokenize(line)
            lexicon+=list(tokens)
    
    with open(neg,"r") as neg_f:
        lines = neg_f.readlines()
        
        for line in lines[:max_lines]:
            tokens=word_tokenize(line)
            lexicon+=list(tokens)
    
    #lemmatize the word like running,runs,ran---run
    lexicon = [lemmatizer.lemmatize(word) for word in lexicon]
    #get unique words with count
    words_count = Counter(lexicon)
    
    w_count_filter = []
    #get most repeative words between 50-1000
    for word_count in words_count:
        if 1000 > words_count[word_count] > 50:
            w_count_filter.append(word_count)
            
    return w_count_filter

In [5]:
def sample_handling(file,lexicon,classification):

    featureset = []
    #read files pos and neg
    with open(file,'r') as f:
        lines = f.readlines()
        
        #tokenize and lemmatize
        for line in lines[:max_lines]:
            tokens = word_tokenize(line.lower())
            tokens = [lemmatizer.lemmatize(word) for word in tokens]
            
            #feaures of lexicon (useful words) 
            #get a feature vector for every line [0,0,0,0,,0,1,0,0,0,1,0.......]
            features = np.zeros(len(lexicon))
            for word in tokens:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1

            features = list(features)
            featureset.append([features,classification])

    return featureset

In [6]:
def create_feature_sets_and_labels(test_size=0.1):
    
    lexicon = create_lexicon(pos_data,neg_data)
    
    features = []
    
    features += sample_handling(pos_data,lexicon,[1,0])
    features += sample_handling(neg_data,lexicon,[0,1])
    
    random.shuffle(features)
    features = np.array(features)
    
    
    testing_size = int(test_size*len(features))

    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x,train_y,test_x,test_y

In [7]:
if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels()
    # if you want to pickle this data:
    with open('C:/Users/Madhu/NLP/Senti/sentiment_set.pickle','wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y],f)